In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import tensorflow as tf
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import xgboost
import sklearn.cluster

In [ ]:
# 데이터 전처리

In [28]:
#충전기 설치현황
file_path = 'c:/Users/oop04/taxio_image/'
p = pathlib.Path(file_path)

설치현황 = pd.read_csv(p.joinpath('01.광양시_충전기설치현황.csv'))
설치현황 = 설치현황[['이용대수', '급속/완속', 'lon', 'lat']]
설치현황['급속/완속'] = np.where(설치현황['급속/완속']=='급속',0,1) #급속 = 0, 완속 = 1
        
설치현황

,이용대수,급속/완속,lon,lat
0,2,0,127.568300,34.963002
1,1,0,127.583323,34.980334
2,5,1,127.689013,34.953759
3,2,1,127.716708,34.964032
4,2,1,127.732440,34.940444
5,1,1,127.582443,34.963334
6,2,1,127.750981,34.962506
7,1,1,127.760072,34.941198
8,2,1,127.725127,34.962147
9,1,1,127.580612,34.973057


In [27]:
#주차장 공간정보
file_path = 'c:/Users/oop04/taxio_image/'
p = pathlib.Path(file_path)

주차장 = pd.read_csv(p.joinpath('02.광양시_주차장_공간정보.csv'))
주차장 = 주차장.set_index("주차장명칭")
주차장 = 주차장[['면적', '주차면수', 'lon', 'lat']]
주차장

,면적,주차면수,lon,lat
주차장명칭,,,,
인동숲앞 주차장,4189.0,161.0,127.586883,34.970324
구터미널주차장,2968.0,125.0,127.695428,34.936252
구경찰서 주차장,2836.0,89.0,127.584586,34.975637
칠성1지구,1980.0,52.0,127.580683,34.970130
칠성1지구,1970.0,64.0,127.580707,34.972523
...,...,...,...,...
포석정위도로변,264.0,10.0,127.604822,35.079214
동곡산장옆,1212.0,60.0,127.613417,35.070673
하천첨주변,2381.0,96.0,127.611774,35.002759


In [29]:
#대중집합시설 야영장
file_path = 'c:/Users/oop04/taxio_image/'
p = pathlib.Path(file_path)

야영장 = pd.read_csv(p.joinpath('05.광양시_대중집합시설_야영장.csv'))
야영장 = 야영장[['소유주체(공공/민간)','lon', 'lat']]
야영장['소유주체(공공/민간)'] = np.where(야영장['소유주체(공공/민간)']=='민간',0,1) #민간 = 0, 공공 = 1
야영장

,소유주체(공공/민간),lon,lat
0,0,127.601787,35.096566
1,0,127.710849,35.060546
2,0,127.631555,35.069553
3,0,127.562487,35.076193
4,1,127.591184,35.054216
5,1,127.581644,35.005226
